In [2]:
import pandas as pd 
import numpy as np

## Introduction

#### Output green_spaces
- dataframe with green spaces in year 2021
- data to different types of green spaces collected and prepared for further analysis in QGIS.
- Data in QGIS was downloaded from Geoportal Berlin using WFS connection/query.
- Different spatial datasets were combined (using spatial join function) in QGIS and then exported as dataframes.
- Dataframes will be imported here in Notebook and prepared for further analysis (modelling) here. 

#### Types of green spaces:
- parks
- allotment gardens
- playgrounds
- cemetries
- forests

#### Aim of analysis
- Expected output:
    * area of subdistrict, which is covered with green spaces
    * Total number for each green spaces type
    * Number of green spaces within district

#### Processing data
- recalculating g_area (from qm to output in ha)
- g_type usign coding:
  * F forests
  * C cemetries
  * A allotment gardens
  * P parks
  * G playgrounds

### Importing dataframes with green spcaes

In [3]:
# Cemetries

cem = pd.read_excel('../Notebook/source/cemetries_lor_tab.xlsx')
cem.head()

,schl5,bezirk,nutz,flalle,BZR_ID,BZR_NAME
0,1801531181000300,Pankow,150,100552.610957,112005,Alt-Hohenschönhausen Süd
1,1801540131000000,Pankow,150,81325.165822,112005,Alt-Hohenschönhausen Süd
2,2201791011000100,Lichtenberg,150,224344.918673,112005,Alt-Hohenschönhausen Süd
3,2201791011000200,Lichtenberg,150,50329.278572,112005,Alt-Hohenschönhausen Süd
4,1200660481000200,Steglitz-Zehlendorf,150,21873.829338,63006,Drakestraße


In [4]:
cem.shape

(219, 6)

In [5]:
cem.isna().sum().sum()

0

In [6]:
cem.dtypes

schl5         int64
bezirk       object
nutz          int64
flalle      float64
BZR_ID        int64
BZR_NAME     object
dtype: object

In [7]:
cem = cem.rename(columns={'schl5':'id',
                          'bezirk':'district',
                         'nutz':'g_type',
                         'flalle':'g_area',
                         'BZR_ID':'lor',
                         'BZR_NAME':'subdistrict'})

In [8]:
cem = cem [["id", "lor", "subdistrict", "district", "g_type", "g_area"]]

In [9]:
# encoding g_type

cem['g_type'].replace({150: 'C'}, inplace=True)

In [10]:
cem["g_type"].value_counts(dropna=False)

g_type
C    219
Name: count, dtype: int64

In [11]:
# recalculating area to ha

cem["g_area"] = (cem["g_area"]/10000).round(2)

In [12]:
cem.head()

,id,lor,subdistrict,district,g_type,g_area
0,1801531181000300,112005,Alt-Hohenschönhausen Süd,Pankow,C,10.06
1,1801540131000000,112005,Alt-Hohenschönhausen Süd,Pankow,C,8.13
2,2201791011000100,112005,Alt-Hohenschönhausen Süd,Lichtenberg,C,22.43
3,2201791011000200,112005,Alt-Hohenschönhausen Süd,Lichtenberg,C,5.03
4,1200660481000200,63006,Drakestraße,Steglitz-Zehlendorf,C,2.19


In [13]:
# Allotment gardens

allotment = pd.read_excel('../Notebook/source/allotment_lor_tab.xlsx')
allotment.head()

,schl5,bezirk,nutz,flalle,BZR_ID,BZR_NAME
0,2201790061000200,Lichtenberg,160,51046.313173,112005,Alt-Hohenschönhausen Süd
1,2201791001000100,Lichtenberg,160,28383.757861,112005,Alt-Hohenschönhausen Süd
2,2201791031000200,Lichtenberg,160,40909.415105,112005,Alt-Hohenschönhausen Süd
3,2201791041000200,Lichtenberg,160,8745.729451,112005,Alt-Hohenschönhausen Süd
4,900460121000200,Charlottenburg-Wilmersdorf,160,10329.118857,44007,Grunewald


In [14]:
allotment.isna().sum().sum()

0

In [15]:
allotment = allotment.rename(columns={'schl5':'id',
                          'bezirk':'district',
                         'nutz':'g_type',
                         'flalle':'g_area',
                         'BZR_ID':'lor',
                         'BZR_NAME':'subdistrict'})

allotment = allotment [["id", "lor", "subdistrict", "district", "g_type", "g_area"]]

allotment['g_type'].replace({160: 'A'}, inplace=True)        # encoding green spaces type
allotment["g_area"] = (allotment["g_area"]/10000).round(2)   # recalculating area to ha

In [16]:
allotment["g_type"].value_counts(dropna=False)

g_type
A    726
Name: count, dtype: int64

In [17]:
allotment.head()

,id,lor,subdistrict,district,g_type,g_area
0,2201790061000200,112005,Alt-Hohenschönhausen Süd,Lichtenberg,A,5.10
1,2201791001000100,112005,Alt-Hohenschönhausen Süd,Lichtenberg,A,2.84
2,2201791031000200,112005,Alt-Hohenschönhausen Süd,Lichtenberg,A,4.09
3,2201791041000200,112005,Alt-Hohenschönhausen Süd,Lichtenberg,A,0.87
4,900460121000200,44007,Grunewald,Charlottenburg-Wilmersdorf,A,1.03


In [18]:
# Allotment gardens

forests = pd.read_excel('../Notebook/source/forests_lor_tab.xlsx')
forests.head()

,schl5,bezirk,nutz,flalle,BZR_ID,BZR_NAME
0,2201791121000000,Lichtenberg,100,1227.648788,112005,Alt-Hohenschönhausen Süd
1,900471111000100,Charlottenburg-Wilmersdorf,100,136364.877818,44007,Grunewald
2,900471111000200,Charlottenburg-Wilmersdorf,100,20622.844725,44007,Grunewald
3,900471111000300,Charlottenburg-Wilmersdorf,100,730.714075,44007,Grunewald
4,900471111000400,Charlottenburg-Wilmersdorf,100,12415.205789,44007,Grunewald


In [19]:
forests.isna().sum().sum()

0

In [20]:
forests = forests.rename(columns={'schl5':'id',
                          'bezirk':'district',
                         'nutz':'g_type',
                         'flalle':'g_area',
                         'BZR_ID':'lor',
                         'BZR_NAME':'subdistrict'})

forests = forests [["id", "lor", "subdistrict", "district", "g_type", "g_area"]]

forests['g_type'].replace({100: 'F'}, inplace=True)        # encoding green spaces type
forests["g_area"] = (forests["g_area"]/10000).round(2)   # recalculating area to ha

In [21]:
forests["g_type"].value_counts(dropna=False)

g_type
F    2743
Name: count, dtype: int64

In [22]:
forests.head(10)

,id,lor,subdistrict,district,g_type,g_area
0,2201791121000000,112005,Alt-Hohenschönhausen Süd,Lichtenberg,F,0.12
1,900471111000100,44007,Grunewald,Charlottenburg-Wilmersdorf,F,13.64
2,900471111000200,44007,Grunewald,Charlottenburg-Wilmersdorf,F,2.06
3,900471111000300,44007,Grunewald,Charlottenburg-Wilmersdorf,F,0.07
4,900471111000400,44007,Grunewald,Charlottenburg-Wilmersdorf,F,1.24
5,900471111000500,44007,Grunewald,Charlottenburg-Wilmersdorf,F,3.39
6,900471111000600,44007,Grunewald,Charlottenburg-Wilmersdorf,F,4.11
7,900471111000700,44007,Grunewald,Charlottenburg-Wilmersdorf,F,0.09
8,900471111000800,44007,Grunewald,Charlottenburg-Wilmersdorf,F,0.92
9,900471111000900,44007,Grunewald,Charlottenburg-Wilmersdorf,F,0.27


In [23]:
# Parks

park = pd.read_excel('../Notebook/source/parks_lor_tab.xlsx')
park.head()

,po1id,bezirkname,objartname,katasterfl,BZR_ID,BZR_NAME
0,00008100:001c92c7,Lichtenberg,10,10736.0,112005,Alt-Hohenschönhausen Süd
1,00008100:001c92c8,Lichtenberg,10,1987.0,112005,Alt-Hohenschönhausen Süd
2,00008100:001c92c9,Lichtenberg,10,3889.0,112005,Alt-Hohenschönhausen Süd
3,00008100:001c92ca,Lichtenberg,10,2623.0,112005,Alt-Hohenschönhausen Süd
4,00008100:001c92cc,Lichtenberg,10,1237.0,112005,Alt-Hohenschönhausen Süd


In [24]:
park.isna().sum().sum()

0

In [25]:
park = park.rename(columns={'po1id':'id',
                          'bezirkname':'district',
                         'objartname':'g_type',
                         'katasterfl':'g_area',
                         'BZR_ID':'lor',
                         'BZR_NAME':'subdistrict'})

park = park [["id", "lor", "subdistrict", "district", "g_type", "g_area"]]

park['g_type'].replace({10: 'P'}, inplace=True)        # encoding green spaces type
park["g_area"] = (park["g_area"]/10000).round(2)   # recalculating area to ha

In [26]:
park["g_type"].value_counts(dropna=False)

g_type
P     2541
32       1
Name: count, dtype: int64

In [27]:
park.shape

(2542, 6)

In [28]:
park.head(2)

,id,lor,subdistrict,district,g_type,g_area
0,00008100:001c92c7,112005,Alt-Hohenschönhausen Süd,Lichtenberg,P,1.07
1,00008100:001c92c8,112005,Alt-Hohenschönhausen Süd,Lichtenberg,P,0.20


In [29]:
# After checking it was an element, which came here by accident, so I will drop this one element. 

park = park[park["g_type"] != 32]
park.shape

(2541, 6)

In [30]:
park = park.reset_index(drop=True)

In [31]:
park.tail()

,id,lor,subdistrict,district,g_type,g_area
2536,00008100:0011e2e1,51001,Hakenfelde,Spandau,P,0.45
2537,00008100:0011e2e2,51001,Hakenfelde,Spandau,P,0.40
2538,00008100:0011e302,51001,Hakenfelde,Spandau,P,0.22
2539,00008100:0011e305,51001,Hakenfelde,Spandau,P,0.18
2540,00008100:0011e32f,51001,Hakenfelde,Spandau,P,0.00


In [32]:
# Playgrounds

playground = pd.read_excel('../Notebook/source/playgrounds_lor_tab.xlsx')
playground.head()

,po1id,bezirkname,objartname,katasterfl,BZR_ID,BZR_NAME
0,00008100:001c92cb,Lichtenberg,20,2912.0,112005,Alt-Hohenschönhausen Süd
1,00008100:001c95aa,Lichtenberg,20,1340.0,112005,Alt-Hohenschönhausen Süd
2,00008100:001c95be,Lichtenberg,20,932.0,112005,Alt-Hohenschönhausen Süd
3,00008100:001c95e2,Lichtenberg,20,1097.0,112005,Alt-Hohenschönhausen Süd
4,00008100:001c95eb,Lichtenberg,20,712.0,112005,Alt-Hohenschönhausen Süd


In [33]:
playground["objartname"].value_counts(dropna=False)

objartname
20    1872
Name: count, dtype: int64

In [34]:
playground.isna().sum()

po1id         0
bezirkname    0
objartname    0
katasterfl    3
BZR_ID        0
BZR_NAME      0
dtype: int64

In [35]:
playground = playground.dropna(subset=['katasterfl'])

In [36]:
playground.shape

(1869, 6)

In [37]:
playground = playground.reset_index(drop=True)

In [38]:
playground.tail()

,po1id,bezirkname,objartname,katasterfl,BZR_ID,BZR_NAME
1864,00008100:0011e2eb,Spandau,20,827.0,51001,Hakenfelde
1865,00008100:0011e2fe,Spandau,20,370.0,51001,Hakenfelde
1866,00008100:0011e2ff,Spandau,20,1750.0,51001,Hakenfelde
1867,00008100:0028dc39,Spandau,20,1401.0,51001,Hakenfelde
1868,00008100:00300efb,Spandau,20,1415.0,51001,Hakenfelde


In [39]:
playground = playground.rename(columns={'po1id':'id',
                          'bezirkname':'district',
                         'objartname':'g_type',
                         'katasterfl':'g_area',
                         'BZR_ID':'lor',
                         'BZR_NAME':'subdistrict'})

playground = playground [["id", "lor", "subdistrict", "district", "g_type", "g_area"]]

playground['g_type'].replace({20: 'G'}, inplace=True)        # encoding green spaces type
playground["g_area"] = (playground["g_area"]/10000).round(2)   # recalculating area to ha

In [40]:
playground.head()

,id,lor,subdistrict,district,g_type,g_area
0,00008100:001c92cb,112005,Alt-Hohenschönhausen Süd,Lichtenberg,G,0.29
1,00008100:001c95aa,112005,Alt-Hohenschönhausen Süd,Lichtenberg,G,0.13
2,00008100:001c95be,112005,Alt-Hohenschönhausen Süd,Lichtenberg,G,0.09
3,00008100:001c95e2,112005,Alt-Hohenschönhausen Süd,Lichtenberg,G,0.11
4,00008100:001c95eb,112005,Alt-Hohenschönhausen Süd,Lichtenberg,G,0.07


#### Concatinating green spaces into one dataframe

In [41]:
green_spaces = pd.concat([cem, allotment, forests, park, playground], axis=0)
green_spaces

,id,lor,subdistrict,district,g_type,g_area
0,1801531181000300,112005,Alt-Hohenschönhausen Süd,Pankow,C,10.06
1,1801540131000000,112005,Alt-Hohenschönhausen Süd,Pankow,C,8.13
2,2201791011000100,112005,Alt-Hohenschönhausen Süd,Lichtenberg,C,22.43
3,2201791011000200,112005,Alt-Hohenschönhausen Süd,Lichtenberg,C,5.03
4,1200660481000200,63006,Drakestraße,Steglitz-Zehlendorf,C,2.19
...,...,...,...,...,...,...
1864,00008100:0011e2eb,51001,Hakenfelde,Spandau,G,0.08
1865,00008100:0011e2fe,51001,Hakenfelde,Spandau,G,0.04
1866,00008100:0011e2ff,51001,Hakenfelde,Spandau,G,0.18
1867,00008100:0028dc39,51001,Hakenfelde,Spandau,G,0.14


#### Output dataframe green_spaces with all green spaces 

In [42]:
green_spaces.to_excel('green_spaces.xlsx', index=False)

In [43]:
green_spaces.to_csv('green_spaces.csv', index=False)

In [44]:
green_spaces = green_spaces.drop_duplicates()
green_spaces.shape

(8098, 6)

#### Grouping informations 
- Expected output:
    * area of subdistrict, which is covered with green spaces
    * Total number for each green spaces type
    * Number of green spaces within district

In [45]:
green_spaces.head()

,id,lor,subdistrict,district,g_type,g_area
0,1801531181000300,112005,Alt-Hohenschönhausen Süd,Pankow,C,10.06
1,1801540131000000,112005,Alt-Hohenschönhausen Süd,Pankow,C,8.13
2,2201791011000100,112005,Alt-Hohenschönhausen Süd,Lichtenberg,C,22.43
3,2201791011000200,112005,Alt-Hohenschönhausen Süd,Lichtenberg,C,5.03
4,1200660481000200,63006,Drakestraße,Steglitz-Zehlendorf,C,2.19


In [46]:
green_spaces_2 = green_spaces

In [47]:
group = green_spaces_2.pivot_table(index= ['lor','subdistrict', 'g_type'], values = ['id','g_area'], aggfunc = {'id':['count'],'g_area':'sum'}).reset_index()

In [48]:
group.tail(10)

,lor,subdistrict,g_type,g_area,id
,,,,sum,count
529,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,G,2.46,11
530,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,P,187.73,40
531,126010,Rollbergesiedlung,A,13.48,4
532,126010,Rollbergesiedlung,G,0.89,3
533,126010,Rollbergesiedlung,P,7.60,4
534,126011,MV Nord,A,6.78,2
535,126011,MV Nord,G,1.53,6
536,126011,MV Nord,P,3.41,5
537,126012,MV Süd,G,0.24,1


In [49]:
group.shape

(539, 5)

#### Total area of green spaces and total number of them

In [50]:
green_spaces_total = green_spaces_2.pivot_table(index= ['lor','subdistrict'],\
                                                    values = ['id', 'g_area'], aggfunc= {'id':'count', 'g_area':'sum'}).reset_index()

green_spaces_total = green_spaces_total.rename(columns={"g_area":"area_objects", "id":"nb_objects"})

green_spaces_total

,lor,subdistrict,area_objects,nb_objects
0,11001,Tiergarten Süd,229.64,54
1,11002,Regierungsviertel,6.37,15
2,11003,Alexanderplatz,56.18,128
3,11004,Brunnenstraße Süd,26.79,26
4,12005,Moabit West,15.22,51
...,...,...,...,...
138,125008,West 3 - Borsigwalde/Freie Scholle,54.45,35
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,251.19,65
140,126010,Rollbergesiedlung,21.97,11
141,126011,MV Nord,11.72,13


#### Numbers of green spaces per type

In [51]:
# Getting only number of g_type "P": parks

nb_parks = green_spaces_2.pivot_table(index= ['lor','subdistrict'],\
                         values = ['g_type'], aggfunc=lambda x: sum(x == 'P')).reset_index()
                    
nb_parks = nb_parks.rename(columns={"g_type":"nb_p"})
nb_parks.head()
# nb_parks.shape

,lor,subdistrict,nb_p
0,11001,Tiergarten Süd,38
1,11002,Regierungsviertel,12
2,11003,Alexanderplatz,78
3,11004,Brunnenstraße Süd,9
4,12005,Moabit West,22


In [52]:
# Getting only number of g_type "F": forests

nb_forests = green_spaces_2.pivot_table(index= ['lor','subdistrict'],\
                         values = ['g_type'], aggfunc=lambda x: sum(x == 'F')).reset_index()
                    
nb_forests = nb_forests.rename(columns={"g_type":"nb_f"})
nb_forests.tail()
# nb_forests.shape

,lor,subdistrict,nb_f
138,125008,West 3 - Borsigwalde/Freie Scholle,1
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,1
140,126010,Rollbergesiedlung,0
141,126011,MV Nord,0
142,126012,MV Süd,0


In [53]:
# Getting only number of g_type "C": cemetries

nb_cemetries = green_spaces_2.pivot_table(index= ['lor','subdistrict'],\
                         values = ['g_type'], aggfunc=lambda x: sum(x == 'C')).reset_index()
                    
nb_cemetries = nb_cemetries.rename(columns={"g_type":"nb_c"})
nb_cemetries.tail()
# nb_cemetries.shape

,lor,subdistrict,nb_c
138,125008,West 3 - Borsigwalde/Freie Scholle,4
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,2
140,126010,Rollbergesiedlung,0
141,126011,MV Nord,0
142,126012,MV Süd,0


In [54]:
# Getting only number of g_type "A": allotment gardens

nb_allotment = green_spaces_2.pivot_table(index= ['lor','subdistrict'],\
                         values = ['g_type'], aggfunc=lambda x: sum(x == 'A')).reset_index()
                    
nb_allotment = nb_allotment.rename(columns={"g_type":"nb_a"})
nb_allotment.head()
# nb_allotment.shape

,lor,subdistrict,nb_a
0,11001,Tiergarten Süd,0
1,11002,Regierungsviertel,0
2,11003,Alexanderplatz,0
3,11004,Brunnenstraße Süd,0
4,12005,Moabit West,0


In [55]:
# Getting only number of g_type "G": playgrounds

nb_playgrounds = green_spaces_2.pivot_table(index= ['lor','subdistrict'],\
                         values = ['g_type'], aggfunc=lambda x: sum(x == 'G')).reset_index()
                    
nb_playgrounds = nb_playgrounds.rename(columns={"g_type":"nb_g"})
nb_playgrounds.tail()
# nb_playgrounds.shape

,lor,subdistrict,nb_g
138,125008,West 3 - Borsigwalde/Freie Scholle,9
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,11
140,126010,Rollbergesiedlung,3
141,126011,MV Nord,6
142,126012,MV Süd,1


### Concatinating data for green spaces

In [56]:
# gs_analysis = pd.concat([green_spaces_total, nb_parks["nb_p"], nb_forests["nb_f"], nb_cemetries["nb_c"], nb_allotment["nb_a"], nb_playgrounds["nb_g"]], axis=1)
# gs_analysis

In [58]:
gs_analysis = green_spaces_total
gs_analysis

,lor,subdistrict,area_objects,nb_objects
0,11001,Tiergarten Süd,229.64,54
1,11002,Regierungsviertel,6.37,15
2,11003,Alexanderplatz,56.18,128
3,11004,Brunnenstraße Süd,26.79,26
4,12005,Moabit West,15.22,51
...,...,...,...,...
138,125008,West 3 - Borsigwalde/Freie Scholle,54.45,35
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,251.19,65
140,126010,Rollbergesiedlung,21.97,11
141,126011,MV Nord,11.72,13


#### Getting total area of each type of green spaces

- allotment gardens

In [59]:
area_a = group[group["g_type"] == "A"]
area_a

,lor,subdistrict,g_type,g_area,id
,,,,sum,count
12,12006,Moabit Ost,A,1.49,1
15,13007,Osloer Straße,A,15.96,7
22,14009,Parkviertel,A,49.41,9
27,14010,Wedding Zentrum,A,1.30,1
33,22002,Tempelhofer Vorstadt,A,8.58,2
...,...,...,...,...,...
512,124006,West 2 - Heiligensee/Konradshöhe,A,26.90,4
521,125008,West 3 - Borsigwalde/Freie Scholle,A,21.05,6
526,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,A,52.91,11


In [60]:
area_a_df = pd.DataFrame()
area_a_df["lor"] = area_a["lor"]
area_a_df["area_a"] = area_a["g_area"]
area_a_df["nb_a"] = area_a["id"]

area_a_df

,lor,area_a,nb_a
12,12006,1.49,1
15,13007,15.96,7
22,14009,49.41,9
27,14010,1.30,1
33,22002,8.58,2
...,...,...,...
512,124006,26.90,4
521,125008,21.05,6
526,125009,52.91,11
531,126010,13.48,4


In [61]:
gs_analysis_2 = gs_analysis.merge(area_a_df, on="lor", how='left')

gs_analysis_2.head(10)

,lor,subdistrict,area_objects,nb_objects,area_a,nb_a
0,11001,Tiergarten Süd,229.64,54,NaN,NaN
1,11002,Regierungsviertel,6.37,15,NaN,NaN
2,11003,Alexanderplatz,56.18,128,NaN,NaN
3,11004,Brunnenstraße Süd,26.79,26,NaN,NaN
4,12005,Moabit West,15.22,51,NaN,NaN
5,12006,Moabit Ost,43.20,59,1.49,1.0
6,13007,Osloer Straße,43.43,45,15.96,7.0
7,13008,Brunnenstraße Nord,50.26,55,NaN,NaN
8,14009,Parkviertel,256.67,80,49.41,9.0
9,14010,Wedding Zentrum,31.83,49,1.30,1.0


- Forests

In [64]:
area_f = group[group["g_type"] == "F"]
area_f_df = pd.DataFrame()
area_f_df["lor"] = area_f["lor"]
area_f_df["area_f"] = area_f["g_area"]
area_f_df["nb_f"] = area_f["id"]

area_f_df

,lor,area_f,nb_f
24,14009,13.56,4
50,25008,0.35,2
55,31001,696.68,85
60,32002,314.04,45
65,32003,3.56,2
...,...,...,...
509,122005,29.85,7
514,124006,1317.21,238
518,124007,235.47,50
523,125008,8.10,1


In [65]:
gs_analysis_3 = gs_analysis_2.merge(area_f_df, on="lor", how='left')
gs_analysis_3

,lor,subdistrict,area_objects,nb_objects,area_a,nb_a,area_f,nb_f
0,11001,Tiergarten Süd,229.64,54,NaN,NaN,NaN,NaN
1,11002,Regierungsviertel,6.37,15,NaN,NaN,NaN,NaN
2,11003,Alexanderplatz,56.18,128,NaN,NaN,NaN,NaN
3,11004,Brunnenstraße Süd,26.79,26,NaN,NaN,NaN,NaN
4,12005,Moabit West,15.22,51,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
138,125008,West 3 - Borsigwalde/Freie Scholle,54.45,35,21.05,6.0,8.10,1.0
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,251.19,65,52.91,11.0,1.97,1.0
140,126010,Rollbergesiedlung,21.97,11,13.48,4.0,NaN,NaN
141,126011,MV Nord,11.72,13,6.78,2.0,NaN,NaN


In [66]:
area_c = group[group["g_type"] == "C"]
area_c_df = pd.DataFrame()
area_c_df["lor"] = area_c["lor"]
area_c_df["area_c"] = area_c["g_area"]
area_c_df["nb_c"] = area_c["id"]

area_c_df

,lor,area_c,nb_c
4,11003,5.87,4
7,11004,12.64,3
16,13007,17.20,3
19,13008,11.09,3
23,14009,44.78,9
...,...,...,...
504,122004,2.16,1
513,124006,9.42,1
517,124007,12.21,2
522,125008,16.55,4


In [70]:
gs_analysis_4 = gs_analysis_3.merge(area_c_df, on="lor", how='left')
gs_analysis_4

,lor,subdistrict,area_objects,nb_objects,area_a,nb_a,area_f,nb_f,area_c,nb_c
0,11001,Tiergarten Süd,229.64,54,NaN,NaN,NaN,NaN,NaN,NaN
1,11002,Regierungsviertel,6.37,15,NaN,NaN,NaN,NaN,NaN,NaN
2,11003,Alexanderplatz,56.18,128,NaN,NaN,NaN,NaN,5.87,4.0
3,11004,Brunnenstraße Süd,26.79,26,NaN,NaN,NaN,NaN,12.64,3.0
4,12005,Moabit West,15.22,51,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
138,125008,West 3 - Borsigwalde/Freie Scholle,54.45,35,21.05,6.0,8.10,1.0,16.55,4.0
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,251.19,65,52.91,11.0,1.97,1.0,6.12,2.0
140,126010,Rollbergesiedlung,21.97,11,13.48,4.0,NaN,NaN,NaN,NaN
141,126011,MV Nord,11.72,13,6.78,2.0,NaN,NaN,NaN,NaN


In [69]:
area_p = group[group["g_type"] == "P"]
area_p_df = pd.DataFrame()
area_p_df["lor"] = area_p["lor"]
area_p_df["area_p"] = area_p["g_area"]
area_p_df["nb_p"] = area_p["id"]

area_p_df

,lor,area_p,nb_p
1,11001,226.75,38
3,11002,5.80,12
6,11003,43.97,78
9,11004,10.33,9
11,12005,11.04,22
...,...,...,...
525,125008,6.75,15
530,125009,187.73,40
533,126010,7.60,4
536,126011,3.41,5


In [71]:
gs_analysis_5 = gs_analysis_4.merge(area_p_df, on="lor", how='left')
gs_analysis_5

,lor,subdistrict,area_objects,nb_objects,area_a,nb_a,area_f,nb_f,area_c,nb_c,area_p,nb_p
0,11001,Tiergarten Süd,229.64,54,NaN,NaN,NaN,NaN,NaN,NaN,226.75,38
1,11002,Regierungsviertel,6.37,15,NaN,NaN,NaN,NaN,NaN,NaN,5.80,12
2,11003,Alexanderplatz,56.18,128,NaN,NaN,NaN,NaN,5.87,4.0,43.97,78
3,11004,Brunnenstraße Süd,26.79,26,NaN,NaN,NaN,NaN,12.64,3.0,10.33,9
4,12005,Moabit West,15.22,51,NaN,NaN,NaN,NaN,NaN,NaN,11.04,22
...,...,...,...,...,...,...,...,...,...,...,...,...
138,125008,West 3 - Borsigwalde/Freie Scholle,54.45,35,21.05,6.0,8.10,1.0,16.55,4.0,6.75,15
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,251.19,65,52.91,11.0,1.97,1.0,6.12,2.0,187.73,40
140,126010,Rollbergesiedlung,21.97,11,13.48,4.0,NaN,NaN,NaN,NaN,7.60,4
141,126011,MV Nord,11.72,13,6.78,2.0,NaN,NaN,NaN,NaN,3.41,5


In [72]:
area_g = group[group["g_type"] == "G"]
area_g_df = pd.DataFrame()
area_g_df["lor"] = area_a["lor"]
area_g_df["area_g"] = area_a["g_area"]
area_g_df["nb_g"] = area_a["id"]

area_g_df

,lor,area_g,nb_g
12,12006,1.49,1
15,13007,15.96,7
22,14009,49.41,9
27,14010,1.30,1
33,22002,8.58,2
...,...,...,...
512,124006,26.90,4
521,125008,21.05,6
526,125009,52.91,11
531,126010,13.48,4


In [73]:
gs_analysis_6 = gs_analysis_5.merge(area_g_df, on="lor", how='left')
gs_analysis_6

,lor,subdistrict,area_objects,nb_objects,area_a,nb_a,area_f,nb_f,area_c,nb_c,area_p,nb_p,area_g,nb_g
0,11001,Tiergarten Süd,229.64,54,NaN,NaN,NaN,NaN,NaN,NaN,226.75,38,NaN,NaN
1,11002,Regierungsviertel,6.37,15,NaN,NaN,NaN,NaN,NaN,NaN,5.80,12,NaN,NaN
2,11003,Alexanderplatz,56.18,128,NaN,NaN,NaN,NaN,5.87,4.0,43.97,78,NaN,NaN
3,11004,Brunnenstraße Süd,26.79,26,NaN,NaN,NaN,NaN,12.64,3.0,10.33,9,NaN,NaN
4,12005,Moabit West,15.22,51,NaN,NaN,NaN,NaN,NaN,NaN,11.04,22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,125008,West 3 - Borsigwalde/Freie Scholle,54.45,35,21.05,6.0,8.10,1.0,16.55,4.0,6.75,15,21.05,6.0
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,251.19,65,52.91,11.0,1.97,1.0,6.12,2.0,187.73,40,52.91,11.0
140,126010,Rollbergesiedlung,21.97,11,13.48,4.0,NaN,NaN,NaN,NaN,7.60,4,13.48,4.0
141,126011,MV Nord,11.72,13,6.78,2.0,NaN,NaN,NaN,NaN,3.41,5,6.78,2.0


In [ ]:
# Filling NaN with 0 (because NaN means, that there is no object like this in this subdistrict)

In [74]:
gs_analysis_6 = gs_analysis_6.replace(np.nan, 0)
gs_analysis_6

,lor,subdistrict,area_objects,nb_objects,area_a,nb_a,area_f,nb_f,area_c,nb_c,area_p,nb_p,area_g,nb_g
0,11001,Tiergarten Süd,229.64,54,0.00,0.0,0.00,0.0,0.00,0.0,226.75,38,0.00,0.0
1,11002,Regierungsviertel,6.37,15,0.00,0.0,0.00,0.0,0.00,0.0,5.80,12,0.00,0.0
2,11003,Alexanderplatz,56.18,128,0.00,0.0,0.00,0.0,5.87,4.0,43.97,78,0.00,0.0
3,11004,Brunnenstraße Süd,26.79,26,0.00,0.0,0.00,0.0,12.64,3.0,10.33,9,0.00,0.0
4,12005,Moabit West,15.22,51,0.00,0.0,0.00,0.0,0.00,0.0,11.04,22,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,125008,West 3 - Borsigwalde/Freie Scholle,54.45,35,21.05,6.0,8.10,1.0,16.55,4.0,6.75,15,21.05,6.0
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,251.19,65,52.91,11.0,1.97,1.0,6.12,2.0,187.73,40,52.91,11.0
140,126010,Rollbergesiedlung,21.97,11,13.48,4.0,0.00,0.0,0.00,0.0,7.60,4,13.48,4.0
141,126011,MV Nord,11.72,13,6.78,2.0,0.00,0.0,0.00,0.0,3.41,5,6.78,2.0


In [75]:
gs_analysis_6.isna().sum().sum()

0

In [77]:
gs_analysis_6

,lor,subdistrict,area_objects,nb_objects,area_a,nb_a,area_f,nb_f,area_c,nb_c,area_p,nb_p,area_g,nb_g
0,11001,Tiergarten Süd,229.64,54,0.00,0.0,0.00,0.0,0.00,0.0,226.75,38,0.00,0.0
1,11002,Regierungsviertel,6.37,15,0.00,0.0,0.00,0.0,0.00,0.0,5.80,12,0.00,0.0
2,11003,Alexanderplatz,56.18,128,0.00,0.0,0.00,0.0,5.87,4.0,43.97,78,0.00,0.0
3,11004,Brunnenstraße Süd,26.79,26,0.00,0.0,0.00,0.0,12.64,3.0,10.33,9,0.00,0.0
4,12005,Moabit West,15.22,51,0.00,0.0,0.00,0.0,0.00,0.0,11.04,22,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,125008,West 3 - Borsigwalde/Freie Scholle,54.45,35,21.05,6.0,8.10,1.0,16.55,4.0,6.75,15,21.05,6.0
139,125009,Nord 2 - Waidmannslust/Wittenau/Lübars,251.19,65,52.91,11.0,1.97,1.0,6.12,2.0,187.73,40,52.91,11.0
140,126010,Rollbergesiedlung,21.97,11,13.48,4.0,0.00,0.0,0.00,0.0,7.60,4,13.48,4.0
141,126011,MV Nord,11.72,13,6.78,2.0,0.00,0.0,0.00,0.0,3.41,5,6.78,2.0


In [78]:
gs_analysis_6.to_excel('green_spaces_summary.xlsx', index=False)